## setup connection, create table

In [14]:
import sys
sys.path.insert(0, '/opt/gpudb/api/python/gpudb/')
sys.path.insert(0, '/opt/gpudb/api/python/gpudb/packages')
import gpudb
import cStringIO
import collections
from avro import schema, io
import os

# init
gpudb_host = "127.0.0.1"
h_db = gpudb.GPUdb(encoding = 'BINARY', host = gpudb_host, port = '9191')

my_type = """
{
    "type": "record",
    "name": "image",
    "fields": [
    {"name":"image","type":"bytes"}
    ]
}""".replace('\n', '').replace(' ', '')   

def creatTable(type_def=my_type,table_name="imageTableInference",type_prop=type_properties,collection="MASTER"):   
    response = h_db.create_type( type_definition=type_def, label = 'image')
    type_id = response['type_id']
    if h_db.has_table(table_name=table_name):
        h_db.clear_table(table_name=table_name)
    response=h_db.create_table( table_name=table_name,type_id=type_id,options={"collection_name":collection})
    
creatTable()

## load the image to database

In [13]:
def ingestImage(imageFolder="./images/",tableName="imageTableInference",collection="MASTER"):
    datum = collections.OrderedDict()
    encoded_obj_list=[]
    listImages=os.listdir(imageFolder)
    for image in listImages:
        myimage=open(imageFolder+image,"r").read()
        datum["image"] = myimage
        encoded_obj_list.append(h_db.encode_datum(my_type, datum))
    #print encoded_obj_list
    response = h_db.insert_records( table_name = tableName,
                                    data = encoded_obj_list,
                                    list_encoding = 'binary',
                                    options = {} )

ingestImage()